In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [123]:
df = pd.read_csv('/content/retail_data.csv')

In [124]:
df.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [125]:
df.shape


(302010, 30)

In [126]:
#dropping some of the least significant features
df.drop(columns =['Transaction_ID','Name',
          'Email', 'Phone','Total_Purchases','Amount','Product_Type'], inplace = True)

adding the full address to a separate column

Standardizing the date timne and merge them in as a single
 feature

In [127]:
from datetime import datetime

def parse_date(date_str):
    if pd.isna(date_str):  # Catches both NaN and NaT
        return pd.NaT

    date_str = str(date_str).strip()

    formats = ['%m-%d-%y', '%m/%d/%Y', '%d-%m-%y', '%Y-%m-%d']
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    return pd.NaT  # return Not-a-Time if no format matches


In [128]:
#parsing the date and time
df['Date'] = df['Date'].astype(str).apply(parse_date)
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce').dt.time

In [129]:
# Combine into DateTime
df['DateTime'] = pd.to_datetime(
    df['Date'].dt.date.astype(str) + ' ' + df['Time'].astype(str),
    errors='coerce'
)

# Drop old columns
df = df.drop(columns=['Date', 'Time', 'Year', 'Month'])

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302010 entries, 0 to 302009
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Customer_ID       301702 non-null  float64       
 1   Address           301695 non-null  object        
 2   City              301762 non-null  object        
 3   State             301729 non-null  object        
 4   Zipcode           301670 non-null  float64       
 5   Country           301739 non-null  object        
 6   Age               301837 non-null  float64       
 7   Gender            301693 non-null  object        
 8   Income            301720 non-null  object        
 9   Customer_Segment  301795 non-null  object        
 10  Total_Amount      301660 non-null  float64       
 11  Product_Category  301727 non-null  object        
 12  Product_Brand     301729 non-null  object        
 13  Feedback          301826 non-null  object        
 14  Ship

In [131]:
# customer_purchases = df.groupby('Customer_ID')['Total_Amount']
# print(customer_purchases.describe())

In [132]:
df.isnull().sum()

,0
Customer_ID,308
Address,315
City,248
State,281
Zipcode,340
Country,271
Age,173
Gender,317
Income,290
Customer_Segment,215


In [133]:
df.dropna(inplace=True)

In [134]:
df.isnull().sum()

,0
Customer_ID,0
Address,0
City,0
State,0
Zipcode,0
Country,0
Age,0
Gender,0
Income,0
Customer_Segment,0


In [135]:
df.head()

,Customer_ID,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Total_Amount,Product_Category,Product_Brand,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products,DateTime
0,37249.0,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular,324.086270,Clothing,Nike,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts,2023-09-18 22:03:55
1,69749.0,82072 Dawn Centers,Nottingham,England,99071.0,UK,19.0,Female,Low,Premium,806.707815,Electronics,Samsung,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab,2023-12-31 08:42:04
2,30192.0,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,48.0,Male,Low,Regular,1063.432799,Books,Penguin Books,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment,2023-04-26 04:06:29
3,62101.0,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,56.0,Male,High,Premium,2466.854021,Home Decor,Home Depot,Excellent,Standard,PayPal,Processing,4.0,Utility knife,2023-05-08 14:55:17
4,27901.0,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,22.0,Male,Low,Premium,248.553049,Grocery,Nestle,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies,2024-01-10 16:54:07


In [136]:
df.shape

(296620, 20)

In [137]:
gender_spending = df.groupby('Gender')['Total_Amount'].mean().sort_values(ascending=False)
print(gender_spending)

Gender
Female    1368.569808
Male      1366.794238
Name: Total_Amount, dtype: float64


In [138]:
# Purchase frequency
df['Purchase_Count'] = df.groupby('Customer_ID')['Customer_ID'].transform('count')

In [139]:
# Average spending
df['Avg_Spend'] = df.groupby('Customer_ID')['Total_Amount'].transform('mean')

In [140]:
# Preferred product category
df['Preferred_Category'] = df.groupby('Customer_ID')['Product_Category'].transform(lambda x: x.mode()[0])

DSFP = Days Since FIrst Purchare

In [141]:
# Time since first purchase (customer loyalty)
df['DSFP'] = (df['DateTime'] - df.groupby('Customer_ID')['DateTime'].transform('min')).dt.days

In [142]:
# Customer value segment
conditions = [
    (df['Total_Amount'] > 600) & (df['Purchase_Count'] > 3),
    (df['Total_Amount'] > 300),
    (df['Total_Amount'] <= 300)
]
choices = ['High Value', 'Medium Value', 'Low Value']
df['Value_Segment'] = np.select(conditions, choices, default='Other Value')

In [143]:
df['Value_Segment'].value_counts()

,count
Value_Segment,
High Value,127581
Medium Value,120412
Low Value,48627


In [144]:
value_encoding = {"High Value": "1", "Medium Value": "2", "Low Value": "3" }
df["Value_Segment"] = df['Value_Segment'].map(value_encoding)

In [145]:
df['Value_Segment'].value_counts()

,count
Value_Segment,
1,127581
2,120412
3,48627


In [146]:
df['Feedback'].value_counts()

,count
Feedback,
Excellent,98952
Good,93496
Average,61571
Bad,42601


In [147]:
feedback_encoding = {"Bad": "1", "Average": "2", "Good": "3", "Excellent": "4" }
df["Feedback"] = df['Feedback'].map(feedback_encoding)

In [148]:
with pd.option_context('display.max_columns', None):
    print(df.head())

   Customer_ID                      Address        City            State  \
0      37249.0            3959 Amanda Burgs    Dortmund           Berlin   
1      69749.0           82072 Dawn Centers  Nottingham          England   
2      30192.0            4133 Young Canyon     Geelong  New South Wales   
3      62101.0  8148 Thomas Creek Suite 100    Edmonton          Ontario   
4      27901.0    5813 Lori Ports Suite 269     Bristol          England   

   Zipcode    Country   Age  Gender Income Customer_Segment  Total_Amount  \
0  77985.0    Germany  21.0    Male    Low          Regular    324.086270   
1  99071.0         UK  19.0  Female    Low          Premium    806.707815   
2  75929.0  Australia  48.0    Male    Low          Regular   1063.432799   
3  88420.0     Canada  56.0    Male   High          Premium   2466.854021   
4  48704.0         UK  22.0    Male    Low          Premium    248.553049   

  Product_Category  Product_Brand Feedback Shipping_Method Payment_Method  \
0  

In [150]:
df.to_csv('processed_retail_data.csv', index=False, encoding='utf-8')